# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

#### Import the mnist dataset from keras datasets

In [0]:
import tensorflow as tf
import numpy as np
import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn import metrics
from sklearn.metrics import classification_report
from tensorflow import keras
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(42)

#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [148]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [149]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [150]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [0]:
x_train_lt5 = x_train_lt5.reshape(len(x_train_lt5),28,28,1)
x_train_gt5 = x_train_gt5.reshape(len(x_train_gt5),28,28,1)
x_test_lt5 = x_test_lt5.reshape(len(x_test_lt5),28,28,1)
x_test_gt5 = x_test_gt5.reshape(len(x_test_gt5),28,28,1)

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [0]:
x_train_lt5 = x_train_lt5/255
x_test_lt5 = x_test_lt5/255

### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [153]:
print('x_train_lt5' ,x_train_lt5.shape)
print('x_train_gt5' , x_train_gt5.shape)
print('x_test_lt5', x_test_lt5.shape)
print('x_test_gt5', x_test_gt5.shape)

x_train_lt5 (30596, 28, 28, 1)
x_train_gt5 (29404, 28, 28, 1)
x_test_lt5 (5139, 28, 28, 1)
x_test_gt5 (4861, 28, 28, 1)


In [154]:
print('X_train shape:', x_train_lt5.shape)
print('X_test shape:', x_test_lt5.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_lt5  = keras.utils.to_categorical(y_train_lt5, 5)
y_test_lt5 = keras.utils.to_categorical(y_test_lt5 , 5)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [0]:
model = Sequential()

#Add a Convolutional Layer with 32 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1),name='conv_1'))

#Add a Convolutional Layer with 64 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(32, (3, 3), activation='relu',name='conv_2'))

#Add a MaxPooling Layer of size 2X2 
model.add(MaxPooling2D(pool_size=(2, 2),name='max_1'))

#Apply Dropout with 0.25 probability 
model.add(Dropout(0.25,name='drop_1'))

### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [0]:
#Flatten the layer
model.add(Flatten())

#Add Fully Connected Layer with 128 units and activation function as 'ReLU'
model.add(Dense(128, activation='relu',name='dense_1'))

#Apply Dropout with 0.5 probability 
model.add(Dropout(0.5,name='drop_2'))

#Add Fully Connected Layer with 10 units and activation function as 'softmax'
model.add(Dense(5, activation='softmax',name='dense_3'))

### Print the training and test accuracy for 5 epochs

In [158]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

#To use adam optimizer for learning weights with learning rate = 0.001
optimizer = Adam(lr=0.001)
#Set the loss function and optimizer for the model training
model.compile(loss=categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])
#Training on the dataset
model.fit(x_train_lt5, y_train_lt5,
          batch_size=100,
          epochs=5,
          verbose=1,
          validation_data=(x_test_lt5, y_test_lt5))
predictions_train = model.predict(x_train_lt5)
predictions_test = model.predict(x_test_lt5)

Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 4s 136us/step - loss: 0.0076 - acc: 0.9972 - val_loss: 0.0066 - val_acc: 0.9981
Epoch 2/5
30596/30596 [==============================] - 4s 124us/step - loss: 0.0051 - acc: 0.9984 - val_loss: 0.0047 - val_acc: 0.9988
Epoch 3/5
30596/30596 [==============================] - 4s 123us/step - loss: 0.0034 - acc: 0.9986 - val_loss: 0.0049 - val_acc: 0.9981
Epoch 4/5
30596/30596 [==============================] - 4s 122us/step - loss: 0.0046 - acc: 0.9983 - val_loss: 0.0044 - val_acc: 0.9988
Epoch 5/5
30596/30596 [==============================] - 4s 122us/step - loss: 0.0047 - acc: 0.9985 - val_loss: 0.0031 - val_acc: 0.9988


In [159]:
print("Train Accuracy: ", metrics.accuracy_score(np.argmax(y_train_lt5, axis=1), np.argmax(predictions_train, axis=1)))
print("Classification Report for train data")
print(classification_report(np.argmax(y_train_lt5, axis=1), np.argmax(predictions_train, axis=1)))
print("Test Accuracy: ", metrics.accuracy_score(np.argmax(y_test_lt5, axis=1), np.argmax(predictions_test, axis=1)))
#print("Classification Report for test data")
print(classification_report(np.argmax(y_test_lt5, axis=1), np.argmax(predictions_test, axis=1)))

Train Accuracy:  0.9999673159890182
Classification Report for train data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5923
           1       1.00      1.00      1.00      6742
           2       1.00      1.00      1.00      5958
           3       1.00      1.00      1.00      6131
           4       1.00      1.00      1.00      5842

    accuracy                           1.00     30596
   macro avg       1.00      1.00      1.00     30596
weighted avg       1.00      1.00      1.00     30596

Test Accuracy:  0.9988324576765908
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       980
           1       1.00      1.00      1.00      1135
           2       1.00      1.00      1.00      1032
           3       1.00      1.00      1.00      1010
           4       1.00      1.00      1.00       982

    accuracy                           1.00      5139
   macro avg       1.00

### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [160]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv_1
conv_1is not trainable

conv_2
conv_2is not trainable

max_1
max_1is not trainable

drop_1
drop_1is not trainable

flatten_1
flatten_1is not trainable

dense_1
dense_1 is trainable

drop_2
drop_2is not trainable

dense_3
dense_3 is trainable



### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [0]:
x_train_gt5 = x_train_gt5/255
x_test_gt5 = x_test_gt5/255
# y_train_gt5 = y_train_gt5/255
# y_test_gt5 = y_test_gt5/255


### Check

Verify the shapes with the given below.

In [162]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404,)
(4861, 28, 28, 1)
(4861,)


## Print the accuracy for classification of digits 5 to 9

In [0]:
y_train_gt5 = keras.utils.to_categorical(y_train_gt5, 5)
y_test_gt5 = keras.utils.to_categorical(y_test_gt5, 5)

In [164]:
#Training on the dataset
config = tf.ConfigProto(
    device_count={'GPU': 1},
    intra_op_parallelism_threads=1,
    allow_soft_placement=True
)

config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6

session = tf.Session(config=config)
with session.as_default():
  with session.graph.as_default():
    model.fit(x_train_gt5, y_train_gt5,
    batch_size=100,
    epochs=20,
    verbose=1,
    validation_data=(x_test_gt5, y_test_gt5))
    predictions_train = model.predict(x_train_gt5)
    predictions_test = model.predict(x_test_gt5)

Train on 29404 samples, validate on 4861 samples
Epoch 1/20


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


29404/29404 [==============================] - 4s 138us/step - loss: 0.1939 - acc: 0.9365 - val_loss: 0.0378 - val_acc: 0.9868
Epoch 2/20
29404/29404 [==============================] - 4s 122us/step - loss: 0.0575 - acc: 0.9822 - val_loss: 0.0312 - val_acc: 0.9899
Epoch 3/20
29404/29404 [==============================] - 4s 121us/step - loss: 0.0384 - acc: 0.9879 - val_loss: 0.0252 - val_acc: 0.9922
Epoch 4/20
29404/29404 [==============================] - 4s 121us/step - loss: 0.0348 - acc: 0.9892 - val_loss: 0.0213 - val_acc: 0.9928
Epoch 5/20
29404/29404 [==============================] - 4s 120us/step - loss: 0.0263 - acc: 0.9921 - val_loss: 0.0180 - val_acc: 0.9944
Epoch 6/20
29404/29404 [==============================] - 4s 123us/step - loss: 0.0221 - acc: 0.9926 - val_loss: 0.0206 - val_acc: 0.9928
Epoch 7/20
29404/29404 [==============================] - 4s 121us/step - loss: 0.0181 - acc: 0.9942 - val_loss: 0.0193 - val_acc: 0.9942
Epoch 8/20
29404/29404 [=====================

# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [169]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [170]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [171]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [0]:
X_train = twenty_train.data
X_test = twenty_test.data
y_train = twenty_train.target
y_test = twenty_test.target
# split the new DataFrame into training and testing sets [Default test size = 25%]
#X_train, X_test, y_train, y_test = train_test_split(twenty_train, twenty_test, random_state=123)
X_train = twenty_train.data
X_test = twenty_test.data
y_train = twenty_train.target
y_test = twenty_test.target
# split the new DataFrame into training and testing sets [Default test size = 25%]
#X_train, X_test, y_train, y_test = train_test_split(twenty_train, twenty_test, random_state=123)

### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [178]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
tfid = TfidfVectorizer(ngram_range=(1, 2),max_df=1.0,min_df=1)
X_train_dtm = tfid.fit_transform(X_train)
print('Features: ', X_train_dtm.shape[1])
X_test_dtm = tfid.transform(X_test)
print('Features: ', X_test_dtm.shape[1])
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train_dtm, y_train)
y_pred_class_LR = logreg.predict(X_test_dtm)
#print(metrics.accuracy_score(y_test, y_pred_class))
y_train_LR = logreg.predict(X_train_dtm)
print('Train Accuracy for LR: ',metrics.accuracy_score(y_train, y_train_LR))
print('Test Accuracy for LR: ',metrics.accuracy_score(y_test, y_pred_class_LR))

Features:  299984
Features:  299984


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Train Accuracy for LR:  1.0
Test Accuracy for LR:  0.9267643142476698


In [0]:
tfidf = TfidfVectorizer(ngram_range=(1, 3),max_df=1.0,min_df=1)
#NB_test(vect)
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_train_pred = nb.predict(X_train_dtm)
y_pred_class = nb.predict(X_test_dtm)

In [180]:
print('Train Accuracy for NB : ', metrics.accuracy_score(y_train,y_train_pred))
print('Test Accuracy for NB: ', metrics.accuracy_score(y_test, y_pred_class))

Train Accuracy for NB :  0.9813912272928667
Test Accuracy for NB:  0.8229027962716379
